Notebook should only be used on remote servers!

TODO:
- Create scripts for orchestrating 
    - Write in reference of local directory
- Remember to keep secret stuff secret
    - Perhaps .gitignore?
- Write up general schemas
- Functions
    - Login
    - Navigation
    - Preparation
        - Upload / Emulate 'Push'
        - Selecting config
        - Creating dirs
    - Running sims
        - Managing / Supervising disk / memory
    - Downloading data
    - Linking to project data model
        - Filling in data model with logs